In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

# 准备数据


class DiabetesDataset(Dataset):
    def __init__(self, path):
        xy = np.loadtxt(path, delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, :-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


my_dataset = DiabetesDataset('diabetes.csv')
train_data = DataLoader(dataset=my_dataset, batch_size=256,
                        shuffle=True, num_workers=0)


# 构建模型
class My_Model(torch.nn.Module):
    def __init__(self):
        super(My_Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 64)
        self.linear2 = torch.nn.Linear(64, 32)
        self.linear3 = torch.nn.Linear(32, 4)
        self.linear4 = torch.nn.Linear(4, 1)
        self.ReLu = torch.nn.ReLU()
        self.Sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.ReLu(self.linear1(x))
        x = self.ReLu(self.linear2(x))
        x = self.ReLu(self.linear3(x))
        x = self.Sigmoid(self.linear4(x))
        return x


model = My_Model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# construct loss and optimizer
criterion = torch.nn.BCELoss(reduction='sum')
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    for epoch in range(1000):
        for i, (input, output) in enumerate(train_data, 0):
            input, output = input.to(device), output.to(device)
            y_pred = model(input)
            loss = criterion(y_pred, output)
            # print("epoch = ", epoch, " loss = ", loss.item())

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()


def test():
    with torch.no_grad():  # 不构建计算图
        test_xy = np.loadtxt('test.csv', delimiter=',', dtype=np.float32)
        test_x_data = torch.from_numpy(test_xy[:, :-1])
        test_y_data = torch.from_numpy(test_xy[:, [-1]])
        test_x_data, test_y_data = test_x_data.to(device), test_y_data.to(device)
        y_test = model(test_x_data)
        y_pred_label = torch.where(
            y_test >= 0.5, torch.tensor([1.0]), torch.tensor([0.0]))
        acc = torch.eq(y_pred_label, test_y_data).sum().item() / \
            test_y_data.size(0)
        print("acc = ", acc)


if __name__ == '__main__':  # windows系统下没这句会报错
    print("SGD training")
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    train()
    test()
    print("Adam training")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    train()
    test()


SGD training
acc =  0.8823529411764706
Adam training
acc =  0.8823529411764706
